## Теоретическая часть


1. Ответьте на вопросы:  
В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?  
Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы  




2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:  
1) Какой датасет используют авторы?  
2) Что используют в качестве признаков?  
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко  

## Практическая часть


In [2]:
import lightfm

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [3]:
# import data
# utils functions like in webinar

### Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

### Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

### LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 17%  
У Вас, скорее всего, возникнет проблема со временем обучения. Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

*задание необязательно


1 Вопрос. Гибридные системы работают с внешними по отношению к матрице интеракций признаками (признаки только user и и только items).
    Примеры: холодный старт (новый юзер, новый товар).

2 Вопрос. Отличия 3: 1 Есть динамические данные - признаки основанные на запросе; 2 дополнительная матрица пользователей и их признаков (парные признаки). 3 Использует пайплайн из моделей для улучшения качества.

3 Вопрос. 
    1. MovieLens + набор тегов Tag Genome tag set
    2. Жанры фильмов, теги Tag Genome tag set
    3. - модель колаборативной фильтрации MF (на матрице интеракций user - items);
       - модель контентной рекомендательной системы LSI-LR (на дополнительных параметрах);
       - гибридная модель LSI-UP (на матрице интеракций user - items с использванием дополнительных параметров)

## Модуль SRC
### import data
### utils functions like in webinar
В папку в которой хранится ноутбук проекта вставил папку SRС. Убедился в наличии файла __init__.py. Он нужен для того чтобы Python увидел модуль. Доюавил функции в utils. 
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
Эта структура добавляет в системный путь путь модуля если его ранее там не было, ориентируясь от родительской директории файла (полагая что папка модуля там же).

Можно все это сделоть проще, если закинуть все функции и классы в один файл XX.py, тогда просто: from XX import <function>
    
По умолчанию как выше импортируем ссылаясь на конкретный файл из модуля, например:
    from src.utils import <function>

## Отбор признаков */ Теория

1 Обертки
    mlxtend - RFE (Recursive feature elimination, рекурсивное удаление признаков): «жадный» алгоритм поиска, который отбирает признаки с помощью рекурсивного определения всё более маленьких наборов признаков. Для LightFM скорее всего неприменим, так как является "жадным" и потребляет много времени/ ресурсов
    
    SMLR (Sparse Multinomial Logistic Regression, разреженная мультиноминальная логистическая регрессия): этот алгоритм реализует l1-регуляризацию с помощью ARD (Automatic relevance determination, автоматическое определение релевантности) в рамках классической мультиноминальной логистической регрессии. Регуляризация определяет важность каждого признака и обнуляет те, которые бесполезны для прогнозирования. Применим.
    
    ARD (Automatic Relevance Determination Regression, регрессия автоматического определения релевантности): модель использует байесовскую гребневую регрессию (Bayesian Ridge Regression). Она сильнее смещает веса коэффициентов в сторону нуля по сравнению, например, с методом наименьших квадратов. Применим. 
 
    Другие примеры алгоритмов регуляризации: Lasso (реализует l1-регуляризацию), гребневая регрессия (реализует l2-регуляризацию), Elastic Net (реализует l1 — и l2-регуляризацию). 
    
2 Фильтры - более подходящи для LightFM, широко используются для работы с данными высокой размерности, когда методы обертки требуют слишком больших вычислительных мощностей.
    Relief: Этот метод случайным образом выбирает из датасета образцы и обновляет значимость каждого признака на основе разницы между выбранным экземпляром и двумя ближайшими к нему объектами того же и противоположного классов. 
    Критерий Фишера (Fisher score): Обычно используется в задачах бинарной классификации.
    Критерий хи-квадрат (Chi-squared score): Проверяет, есть ли значимая разница между наблюдаемой и ожидаемой частотами двух категориальных переменных. Таким образом, проверяется нулевая гипотеза об отсутствии связи между двумя переменными. Применим.
    CFS (Correlation-based feature selection, отбор признаков на основе корреляции). хорошее подмножество признаков содержит такие признаки, которые высоко коррелируют с целевой переменной, и при этом не коррелируют друг с другом. Применим.
    FCBF (Fast correlation-based filter, быстрый фильтр на основе корреляции): Этот метод работает быстрее и эффективнее, чем ReliefF и CFS, и поэтому чаще используется для входных данных высокой размерности. По сути, этот типичный подход, учитывающий релевантность и избыточность, в рамках которого сначала для всех признаков вычисляются Symmetrical Uncertainty (взаимная информация между X и Y I(X, Y), деленная на сумму их энтропий), затем признаки сортируются по этому критерию, а потом удаляются избыточные.
    
3 Методы без учителя
    Дисперсия: было показано, что оценка дисперсии признака может быть эффективным способом отбора признаков. Как правило признаки с почти нулевой дисперсией не являются значимыми, и их можно удалить.
    Средняя абсолютная разность: Вычисляем среднюю абсолютную разность между значениями признака и его средним значением.
    Более высокие значения, как правило, имеют более высокую предсказательную силу.
    Соотношение дисперсий: Среднее арифметическое, деленное на среднее геометрическое. Более высокая дисперсия соответствует более релевантным признакам
    Критерий Лапласа (Laplacian Score)
    Критерий Лапласа в сочетании с энтропией на основе расстояния: в основе алгоритма лежит критерий Лапласа, где кластеризация методом k-средних заменяется на энтропию. 
    MCFS (Multi-Cluster Feature selection, многокластерный отбор признаков): для измерения корреляции между разными признаками выполняется спектральный анализ. Для кластеризации данных и оценки признаков используются собственные вектора  оператора Лапласа(graph Laplacian). 
    Алгоритмы LFSBSS (Localised feature selection, отбор локализованных признаков), взвешенные k-средние (weighted k-means), SPEC и Apriori.

4 Гибридные методы

Другой способ реализации отбора признаков представляет собой гибрид из фильтров и обёрток, объединённых в двухфазный процесс: сначала признаки фильтруются по статистическим свойствам, а затем применяются методы обертки.
